# LAPD 'Part I' crimes database: 2010-present

### Import data tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
from fbprophet import Prophet
import glob
import os

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Read crimes CSV downloaded from LA City data portal

In [4]:
# data exported from '00-lapd-crimes-processing.ipynb'
crimes = pd.read_csv(
    "/Users/mhustiles/data/data/LA/crimes.csv",
    dtype={
        "area_name": str,
        "rpt_dist_no": str,
        "weapon_used_cd": str,
        "crm_cd": str,
        "premises_code": str,
        "division": str,
    },
)

In [5]:
len(
    crimes[crimes["modus_operandi_code"].fillna("").str.contains("2019")].sort_values(
        "date_occurred", ascending=False
    )
)

74

In [6]:
crimes[crimes["modus_operandi_code"].fillna("").str.contains("2019")][["year"]].value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [ ]:
pets = crimes[crimes["modus_operandi_code"].fillna("").str.contains("2019")]

In [ ]:
pets.year.value_counts()

---

### Understanding LAPD's modus operandi codes

In [ ]:
mocrimes = crimes.dropna(subset=["modus_operandi_code"])

In [ ]:
mocodes = pd.read_csv("mo_codes.csv")
mocodes.head()

### Isolating cases involving specific 'MO' tags (ie homeless, , gang, etc)

In [ ]:
# For example...
mocodes_homeless = mocodes[
    mocodes["mo_code_description"].str.lower().str.contains("gang")
]
mocodes_homeless.head()

---

### Parsing codes

In [ ]:
df = crimes[["record_id", "modus_operandi_code"]].copy()

In [ ]:
df["modus_operandi_code"] = (
    df["modus_operandi_code"].dropna().apply(lambda x: x.split(" "))
)

In [ ]:
final = (
    df["modus_operandi_code"]
    .apply(pd.Series)
    .merge(df, left_index=True, right_index=True)
    .drop("modus_operandi_code", axis=1)
    .melt(
        id_vars=["record_id"], value_name="modus_operandi_code", var_name="modoporder"
    )
    .dropna(subset=["modus_operandi_code"])
    .sort_values("record_id")
)

In [ ]:
mo_counts = (
    final.groupby(["modus_operandi_code"])
    .agg("size")
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

In [ ]:
merged_mo_counts = pd.merge(
    mo_counts, mocodes, left_on="modus_operandi_code", right_on="mo_code", how="left"
)

In [ ]:
merged_mo_counts.head()

In [ ]:
merged_mo_counts[
    merged_mo_counts["mo_code_description"].fillna("").str.contains("Animal")
]

---